In [ ]:
!pip install simpy

In [ ]:
import simpy
import random
import math

In [ ]:
Total_Time = 100000

RANDOM_SEED = 978
random.seed(RANDOM_SEED)

hospital_treating_time_rate = 6
home_treating_time_rate = 10


num_of_individuals = 0
S = 147 + 177 + 207
if(S > 1000): num_of_individuals = S
elif(S > 10): num_of_individuals = S + 1000
else: num_of_individuals = S*300

num_of_beds = math.ceil(num_of_individuals/12)
exponential_rate = num_of_individuals/300

num_of_patients = math.ceil(random.expovariate(1.0/exponential_rate))
print(num_of_individuals, num_of_patients)

1531 23


Define the necessary set of arrays for bookkeeping

In [ ]:
service_times = [] #Duration of the conversation between the customer and the operator (Service time)

In [ ]:
class Patient(object):
    def __init__(self, name, env, opr, isInitial):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.r = random.uniform(1,2)
        self.hospital2home_treating_time_rate = 6*self.r
        self.isInitial = isInitial
        self.action = env.process(self.try_to_heal())
    
    
    def try_to_heal(self):
        if(not self.isInitial and self.is_going_hospital() == False):
            print('%s at home at %g' % (self.name, math.ceil(self.env.now)))
            yield self.env.process(self.healing())
            print('%s is healed at %g in home' % (self.name, math.ceil(self.env.now)))
        else:
            print('%s arrives hospital at %g' % (self.name, math.ceil(self.env.now)))
            with bed.request() as req:
                yield req
                print('%s is assigned to a bed at %g' % (self.name, math.ceil(self.env.now)))
                yield self.env.process(self.healing())
                print('%s is healed at %g' % (self.name, math.ceil(self.env.now)))
            
            
    def healing(self):
        if(self.is_going_hospital and bed.count != num_of_beds):
            duration = random.expovariate(1.0/hospital_treating_time_rate)
        elif(self.is_going_hospital):
            print("%s returns home." %(self.name))
            duration = random.expovariate(1.0/self.hospital2home_treating_time_rate)
        else:
            duration = random.expovariate(1.0/home_treating_time_rate)
        yield self.env.timeout(duration)
        service_times.append(duration)
        
        
    def is_going_hospital(self):
        random_number = random.random()
        if(random_number > 0.2): return False
        else: return True
    

In [ ]:
def patient_generator(env, bed):
    """Generate new patients with using exponential rate!!!."""
    for i in range(num_of_patients): #while a çevir.
        yield env.timeout(random.expovariate(exponential_rate))
        patient = Patient('Patient %s' %(i+1), env, bed, False)  

In [ ]:
def patient_generator_half_full(env, bed):
    for i in range (num_of_beds // 2):
        yield env.timeout(0)
        patient = Patient('Patient initial %s' %(i+1), env, bed, True)
    for i in range(num_of_patients):
        yield env.timeout(random.expovariate(exponential_rate))
        patient = Patient('Patient %s' %(i+1), env, bed, False)  

In [ ]:
def patient_generator_full(env, bed):
    for i in range (num_of_beds):
        yield env.timeout(0)
        patient = Patient('Patient initial %s' %(i+1), env, bed, True)
    for i in range(num_of_patients):
        yield env.timeout(random.expovariate(exponential_rate))
        patient = Patient('Patient %s' %(i+1), env, bed, False)  

In [ ]:
env = simpy.Environment()
bed = simpy.Resource(env, capacity = num_of_beds)
env.process(patient_generator_full(env, bed))
env.run(until = Total_Time)

Patient initial 1 arrives hospital at 0
Patient initial 2 arrives hospital at 0
Patient initial 1 is assigned to a bed at 0
Patient initial 3 arrives hospital at 0
Patient initial 2 is assigned to a bed at 0
Patient initial 4 arrives hospital at 0
Patient initial 3 is assigned to a bed at 0
Patient initial 5 arrives hospital at 0
Patient initial 4 is assigned to a bed at 0
Patient initial 6 arrives hospital at 0
Patient initial 5 is assigned to a bed at 0
Patient initial 7 arrives hospital at 0
Patient initial 6 is assigned to a bed at 0
Patient initial 8 arrives hospital at 0
Patient initial 7 is assigned to a bed at 0
Patient initial 9 arrives hospital at 0
Patient initial 8 is assigned to a bed at 0
Patient initial 10 arrives hospital at 0
Patient initial 9 is assigned to a bed at 0
Patient initial 11 arrives hospital at 0
Patient initial 10 is assigned to a bed at 0
Patient initial 12 arrives hospital at 0
Patient initial 11 is assigned to a bed at 0
Patient initial 13 arrives hosp

In [ ]:
print(service_times)

[0.03586171964128584, 0.12786088103587684, 0.12855610435727868, 0.22589314964950116, 0.2729295670532327, 0.3398495874489467, 0.35754142780844467, 0.3961432363110857, 0.4542767778207623, 0.5274699731233299, 0.5395018342564907, 0.5504509291626329, 0.5816818582693353, 0.6394399349622353, 0.763292174611945, 0.7973068856516033, 0.8390762047593283, 1.0782915670072117, 1.1292282626788435, 1.2995983139382299, 1.3574150600397972, 1.4527597235052887, 1.5291931662145073, 1.5557645042899813, 1.6932811813793327, 1.915749036818645, 2.091064978305035, 2.131818946649559, 2.1786910838667426, 2.233128072592993, 2.235684504711231, 2.4857261556314496, 2.5045440740582423, 2.5357802111726504, 2.5700611525207857, 2.6400465926162595, 2.6416273671216697, 2.7119773872543784, 2.8142652118491958, 2.8455719866444107, 2.8477896730697156, 2.877034275308843, 3.058656774641376, 3.2375149750463854, 3.2536455184808655, 3.27562151492674, 3.432674235011879, 3.5277922734623317, 3.534479552549958, 3.55324863194173, 3.649363